In [ ]:
#CAROLINA CAMPOS
#ALEXANDRA RIVERA

In [1]:
from IPython.display import display, Image
from openai import OpenAI
import os
import pandas as pd
import json
import io
from PIL import Image
import requests

In [2]:
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "")) 

#Lets import some helper functions for assistants from https://cookbook.openai.com/examples/assistants_api_overview_python

# Function to display a JSON File
def show_json(obj):
    display(json.loads(obj.model_dump_json()))

# you can change the model if you are on text only
def submit_message(assistant_id, thread, user_message, model=None, file_ids=None):
    if model: 
        client.beta.assistants.update(assistant_id=assistant_id, model=model)

    params = {
        'thread_id': thread.id,
        'role': 'user',
        'content': user_message,
        }
    
    if file_ids:
        #client.beta.assistants.update(assistant_id=assistant_id, tool_resources={"code_interpreter":{"file_ids":file_ids}})        
        params['attachments'] = [{ "file_id": fid, "tools": [{"type": "code_interpreter"}]} for fid in file_ids]
        
    client.beta.threads.messages.create(**params) # create a thread
    return client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant_id)

def get_response(thread):
    return client.beta.threads.messages.list(thread_id=thread.id)

In [3]:
financial_data_path = 'data_financiera.json'
financial_data = pd.read_json(financial_data_path)
financial_data.head(5)

,Year,Quarter,Distribution channel,Revenue ($M),Costs ($M),Customer count,Time
0,2021,Q1,Online Sales,1.50,1.301953,150,2021 Q1
1,2021,Q1,Direct Sales,1.50,1.380809,151,2021 Q1
2,2021,Q1,Retail Partners,1.50,1.348246,152,2021 Q1
3,2021,Q2,Online Sales,1.52,1.308608,152,2021 Q2
4,2021,Q2,Direct Sales,1.52,1.413305,153,2021 Q2


In [4]:
file = client.files.create(
  file=open('data_financiera.json',"rb"),
  purpose='assistants',
)

In [5]:
assistant = client.beta.assistants.create(
  instructions="Eres un asistente de ciencia de datos. Cada vez que recibas un query, escribe el codigo que corresponde y crea una visualización apropiada.",
  name = "data_science_ppt_creator",
  model="gpt-4-1106-preview",
  tools=[{"type": "code_interpreter"}], 
  # tool_resources={
  #     "code_interpreter": {
  #         "file_ids":[file.id]
  #     }
  # }
)

In [6]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "Del archivo .json cargado: Calcular la utilidad (ingreso - gasto)  por cuarto y por año, visualizalo en un grafico de lineas a traves de la distribución de canales, donde los colores de las lineas son rojo, verde, and azul claro",
      "attachments": [{"file_id": file.id, "tools": [{"type": "code_interpreter"}]}]
    }
  ]
)

In [7]:
run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant.id)

In [8]:
#client.beta.threads.delete(run.thread_id)

In [9]:
messages = client.beta.threads.messages.list(thread_id=thread.id)

In [10]:
messages

SyncCursorPage[Message](data=[Message(id='msg_nSczfZkyui5h5CnmlHO6HBIj', assistant_id=None, attachments=[Attachment(file_id='file-Tc16LaqeMnq18ZwOhoMLOeUr', tools=[CodeInterpreterTool(type='code_interpreter')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Del archivo .json cargado: Calcular la utilidad (ingreso - gasto)  por cuarto y por año, visualizalo en un grafico de lineas a traves de la distribución de canales, donde los colores de las lineas son rojo, verde, and azul claro'), type='text')], created_at=1714682645, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_mI8s84l8LGDccJAJ9OB7PZ0W')], object='list', first_id='msg_nSczfZkyui5h5CnmlHO6HBIj', last_id='msg_nSczfZkyui5h5CnmlHO6HBIj', has_more=False)

In [11]:
import time

while True:
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    try:
        #See if image has been created
        messages.data[0].content[0].image_file
        #Sleep to make sure run has completed
        time.sleep(5)
        print('Plot created!')
        break
    except:
        time.sleep(10)
        print('Assistant still working...')

Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Assistant still working...
Plot created!


In [12]:
# to stop the runs
#client.beta.threads.runs.cancel(run_id=run.id, thread_id=thread.id)

In [13]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
[message.content[0] for message in messages.data]

[ImageFileContentBlock(image_file=ImageFile(file_id='file-gCwHSRnhGDXMoPKrwaWUePbT'), type='image_file'),
 TextContentBlock(text=Text(annotations=[], value='Los datos se han importado correctamente a un DataFrame de Pandas, lo que facilitará el cálculo y la visualización. El siguiente paso es calcular la utilidad para cada fila restando el gasto del ingreso, y luego agrupar los datos por año, cuarto y canal de distribución para visualizar las líneas de utilidad por canal de distribución y los colores requeridos.\n\nProcederé a realizar estos cálculos y luego crear el gráfico de líneas.'), type='text'),
 TextContentBlock(text=Text(annotations=[], value="El objeto JSON cuenta con una estructura de diccionario en la que cada clave representa una columna distinta que contiene datos relevantes (como 'Year', 'Quarter', 'Distribution channel', 'Income', 'Expense', 'Gross margin', 'Customer count', y 'Time'). Cada clave tiene un diccionario asociado con índices numerados que parecen representa

In [14]:
# Quick helper function to convert our output file to a png
def convert_file_to_png(file_id, write_path):
    data = client.files.content(file_id)
    data_bytes = data.read()
    with open(write_path, "wb") as file:
        file.write(data_bytes)

In [15]:
plot_file_id = messages.data[0].content[0].image_file.file_id
image_path = "grafica_financiera.png"
convert_file_to_png(plot_file_id,image_path)

#Upload
plot_file = client.files.create(
  file=open(image_path, "rb"),
  purpose='assistants'
)

In [16]:
submit_message(assistant_id=assistant.id, 
               thread=thread, 
               model="gpt-4-turbo", 
               file_ids=None,
               user_message="Escribe dos oraciones de largo aproximadamente medio (~20-30 palabras por oracion) de \
lo mas importante captado de la gráfica que acabas de crear. Esta grafica será añadida para un  conjunto de diapositivas, \
y ellas deberian representar acerca del '¿Que hay detras de los datos?'.")

Run(id='run_AWlI4254vrPWuUBwAEAsJaIr', assistant_id='asst_rkXIKJhqTuUdygLi6udzLFj9', cancelled_at=None, completed_at=None, created_at=1714682815, expires_at=1714683415, failed_at=None, incomplete_details=None, instructions='Eres un asistente de ciencia de datos. Cada vez que recibas un query, escribe el codigo que corresponde y crea una visualización apropiada.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4-turbo', object='thread.run', required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_mI8s84l8LGDccJAJ9OB7PZ0W', tool_choice='auto', tools=[CodeInterpreterTool(type='code_interpreter')], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})

In [17]:
# Hard coded wait for a response, as the assistant may iterate on the bullets.
time.sleep(10)
response = get_response(thread)
bullet_points = response.data[0].content[0].text.value
print(bullet_points)

En la gráfica, se observa que las ventas en línea ('Online Sales') muestran una tendencia creciente de utilidades año tras año, lo que indica una expansión robusta y exponencial de este canal.
Por otro lado, 'Direct Sales' y 'Retail Partners' experimentan fluctuaciones menores en su utilidad, sugiriendo una estabilidad en estos canales que podría reflejar estrategias de marketing y ventas consistentes a lo largo del tiempo.


In [18]:
submit_message(assistant_id=assistant.id, thread=thread, model="gpt-4-turbo", user_message="Dada la gráfica y las viñetas que has creado,\
 piense en un titulo muy breve para una diapositiva. Debe reflejar solo las ideas principales que se te ocurrieron."
)

Run(id='run_KTTCwSMiknVmppQZtrywwmxj', assistant_id='asst_rkXIKJhqTuUdygLi6udzLFj9', cancelled_at=None, completed_at=None, created_at=1714682860, expires_at=1714683460, failed_at=None, incomplete_details=None, instructions='Eres un asistente de ciencia de datos. Cada vez que recibas un query, escribe el codigo que corresponde y crea una visualización apropiada.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4-turbo', object='thread.run', required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_mI8s84l8LGDccJAJ9OB7PZ0W', tool_choice='auto', tools=[CodeInterpreterTool(type='code_interpreter')], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})

In [19]:
#Wait as assistant may take a few steps
time.sleep(10)
response = get_response(thread)
title = response.data[0].content[0].text.value
print(title)

"Tendencias de Utilidad en Diversos Canales de Distribución: Crecimiento y Estabilidad"


In [20]:
#3. DALL·E-3 title image

company_summary = "MiEmpresaFinanciera es una empresa prominente de hardware que manufactura y vende procesadores, tarjetas gráficas\
 y otras piezas esenciales de hardware de computadoras."

In [21]:
response = client.images.generate(
  model='dall-e-3',
  prompt=f"Dado este resumen de la empresa {company_summary}, crea una foto inspiradora que \
  muestre el crecimiento y el camino a seguir. Esto se utilizará en una reunión trimestral \
  de planificación financiera",
       size="1024x1024",
       quality="hd",
       n=1
)
image_url = response.data[0].url

In [22]:
dalle_img_path = 'imagen_dalle3.png'
img = requests.get(image_url)

#Save locally
with open(dalle_img_path,'wb') as file:
  file.write(img.content)

#Upload
dalle_file = client.files.create(
  file=open(dalle_img_path, "rb"),
  purpose='assistants'
)

In [24]:
#4. Creating the slides

title_template = """
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_PARAGRAPH_ALIGNMENT
from pptx.dml.color import RGBColor

# Create a new presentation object
prs = Presentation()

# Add a blank slide layout
blank_slide_layout = prs.slide_layouts[6]
slide = prs.slides.add_slide(blank_slide_layout)

# Set the background color of the slide to black
background = slide.background
fill = background.fill
fill.solid()
fill.fore_color.rgb = RGBColor(0, 0, 0)

# Add image to the left side of the slide with a margin at the top and bottom
left = Inches(0)
top = Inches(0)
height = prs.slide_height
width = prs.slide_width * 3/5
pic = slide.shapes.add_picture(image_path, left, top, width=width, height=height)

# Add title text box positioned higher
left = prs.slide_width * 3/5
top = Inches(2)
width = prs.slide_width * 2/5
height = Inches(1)
title_box = slide.shapes.add_textbox(left, top, width, height)
title_frame = title_box.text_frame
title_p = title_frame.add_paragraph()
title_p.text = title_text
title_p.font.bold = True
title_p.font.size = Pt(38)
title_p.font.color.rgb = RGBColor(255, 255, 255)
title_p.alignment = PP_PARAGRAPH_ALIGNMENT.CENTER

# Add subtitle text box
left = prs.slide_width * 3/5
top = Inches(3)
width = prs.slide_width * 2/5
height = Inches(1)
subtitle_box = slide.shapes.add_textbox(left, top, width, height)
subtitle_frame = subtitle_box.text_frame
subtitle_p = subtitle_frame.add_paragraph()
subtitle_p.text = subtitle_text
subtitle_p.font.size = Pt(22)
subtitle_p.font.color.rgb = RGBColor(255, 255, 255)
subtitle_p.alignment = PP_PARAGRAPH_ALIGNMENT.CENTER
"""

data_vis_template = """
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_PARAGRAPH_ALIGNMENT
from pptx.dml.color import RGBColor

# Create a new presentation object
prs = Presentation()

# Add a blank slide layout
blank_slide_layout = prs.slide_layouts[6]
slide = prs.slides.add_slide(blank_slide_layout)

# Set the background color of the slide to black
background = slide.background
fill = background.fill
fill.solid()
fill.fore_color.rgb = RGBColor(0, 0, 0)

# Define placeholders
image_path = data_vis_img
title_text = "Maximizando Beneficios: El Dominio de las Ventas Activas & Optimización de Venta Directa"
bullet_points = "• Las ventas online consistentemente llevan a ganancias en los cuartos, indicando una fuerte presencia de marketing digital.\n• Las ventas directas muestran fluctuaciones, sugiriendo desempeño variable y la necesidas de apuntar mejoras en el canal."

# Add image placeholder on the left side of the slide
left = Inches(0.2)
top = Inches(1.8)
height = prs.slide_height - Inches(3)
width = prs.slide_width * 3/5
pic = slide.shapes.add_picture(image_path, left, top, width=width, height=height)

# Add title text spanning the whole width
left = Inches(0)
top = Inches(0)
width = prs.slide_width
height = Inches(1)
title_box = slide.shapes.add_textbox(left, top, width, height)
title_frame = title_box.text_frame
title_frame.margin_top = Inches(0.1)
title_p = title_frame.add_paragraph()
title_p.text = title_text
title_p.font.bold = True
title_p.font.size = Pt(28)
title_p.font.color.rgb = RGBColor(255, 255, 255)
title_p.alignment = PP_PARAGRAPH_ALIGNMENT.CENTER

# Add hardcoded "Key Insights" text and bullet points
left = prs.slide_width * 2/3
top = Inches(1.5)
width = prs.slide_width * 1/3
height = Inches(4.5)
insights_box = slide.shapes.add_textbox(left, top, width, height)
insights_frame = insights_box.text_frame
insights_p = insights_frame.add_paragraph()
insights_p.text = "Key Insights:"
insights_p.font.bold = True
insights_p.font.size = Pt(24)
insights_p.font.color.rgb = RGBColor(0, 128, 100)
insights_p.alignment = PP_PARAGRAPH_ALIGNMENT.LEFT
insights_frame.add_paragraph()


bullet_p = insights_frame.add_paragraph()
bullet_p.text = bullet_points
bullet_p.font.size = Pt(12)
bullet_p.font.color.rgb = RGBColor(255, 255, 255)
bullet_p.line_spacing = 1.5
"""

In [25]:
title_text = "MiEmpresaFinanciera"
subtitle_text = "Reunion de Planificacion del Q3, 2023"

In [26]:
run = submit_message(assistant_id=assistant.id, thread=thread, model="gpt-4-turbo", user_message=f"IMPORTANTE: Utilizar {dalle_file.filename} \
con key_id {dalle_file.id}; utilizar {plot_file.filename} con key_id {plot_file.id} en filmina 2. Utilice la plantilla de código incluida \
para crear una diapositiva PPTX que siga el formato de la plantilla, pero utilice la imagen, el nombre/título de la empresa y el nombre/subtítulo \
del documento incluidos:{title_template}. IMPORTANTE: utilice el archivo de imagen incluido en este mensaje como imagen image_path en esta primera \
diapositiva y utilice el nombre de la empresa {title_text} como variable title_text y  utilice subtitle_text {subtitle_text} como \
variable subtitle_text. A CONTINUACIÓN, cree una SEGUNDA diapositiva usando la siguiente plantilla de código: {data_vis_template} para crear \
una diapositiva PPTX que siga el formato de la plantilla, pero use el nombre/título de la empresa y el nombre/subtítulo del \
documento incluidos: {data_vis_template}. IMPORTANTE: use la imagen del diagrama de líneas, que es la segunda imagen adjunta en \
este mensaje, que creó anteriormente en thread como imagen data_vis_img, y use el título de visualización de datos que creó anteriormente \
para la variable title_text, y las viñetas de conocimientos que creó anteriormente para la variable bullet_points. \
Imprima estas DOS DIAPOSITIVAS como un archivo .pptx. Asegúrese de que el resultado sean dos diapositivas, y que cada diapositiva coincida \
con la plantilla respectiva proporcionada en este mensaje.",
file_ids=[plot_file.id, dalle_file.id]
)
run

Run(id='run_Bi1tjjhkumtqUwcaL6W8ONu9', assistant_id='asst_rkXIKJhqTuUdygLi6udzLFj9', cancelled_at=None, completed_at=None, created_at=1714683104, expires_at=1714683704, failed_at=None, incomplete_details=None, instructions='Eres un asistente de ciencia de datos. Cada vez que recibas un query, escribe el codigo que corresponde y crea una visualización apropiada.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4-turbo', object='thread.run', required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_mI8s84l8LGDccJAJ9OB7PZ0W', tool_choice='auto', tools=[CodeInterpreterTool(type='code_interpreter')], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={})

In [27]:
# cancelar la corrida
#client.beta.threads.runs.cancel(run_id=run.id, thread_id=thread.id)

In [28]:
#May take 1-3 mins
while True:
    try:
        response = get_response(thread)
        pptx_id = response.data[0].content[0].text.annotations[0].file_path.file_id
        print("Successfully retrieved pptx_id:", pptx_id)
        break
    except Exception as e:
        print("Assistant still working on PPTX...")
        time.sleep(10)

Assistant still working on PPTX...
Assistant still working on PPTX...
Assistant still working on PPTX...
Assistant still working on PPTX...
Assistant still working on PPTX...
Successfully retrieved pptx_id: file-jgmWUX7r42Bs5RIIMZPZsPCF


In [29]:
# para detener la tarea
#client.beta.threads.delete(thread_id=thread.id)

In [30]:
pptx_id = response.data[0].content[0].text.annotations[0].file_path.file_id
ppt_file= client.files.content(pptx_id)
file_obj = io.BytesIO(ppt_file.read())
with open("filminas_creadas.pptx", "wb") as f:
    f.write(file_obj.getbuffer())

In [31]:
#client.beta.threads.delete(thread_id=)
#client.beta.threads.runs.cancel(run_id=)
#client.beta.assistants.delete(assistant_id=asst_VHenAVYAorsqJ9bgtXDh1603)